# Local Langchain Llama2

In [ ]:
# install modules
#!pipenv install langchain

In [ ]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

In [ ]:
llm.invoke("How can langsmith help with testing?")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ('system', 'You are a very experienced technical writer.'),
    ('user', '{input}')
])

In [ ]:
chain = prompt | llm

In [ ]:
chain.invoke({"input": "how can langsmith help with testing?"})

In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [ ]:
chain = prompt | llm | output_parser

In [ ]:
chain.invoke({"input": "how can langsmith help with testing?"})

In [ ]:
#!pipenv install beatifulsoup4
#!pipenv install pypdf

In [ ]:
# from langchain_community.document_loaders import WebBaseLoader
# loader = WebBaseLoader("https://docs.smith.langchain.com/overview")

# docs = loader.load()

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("TheProblemsWithBuildingYourOwnCopilot.pdf")
pages = loader.load_and_split()

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

In [ ]:
#!pipenv install faiss-cpu

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(pages)
vector = FAISS.from_documents(documents, embeddings)

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "What are the challenges of building Copilot?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

In [ ]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
response = retrieval_chain.invoke({"input": "What are the challenges of building a product's custom Copilot?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can a Copilot be built carefully?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [ ]:
chat_history = [HumanMessage(content="Can a Copilot be built carefully?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})